# SBC - Pràctica CBR

- Aina Gomila
- Ruth Parajó
- Olivia Puig
- Marc Ucelayeta

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from cbr import CBR
from utils import Usuari, Llibre

In [3]:
att = {
    "edat": 20,
    "estil_literari": "realisme",
    "temas_especifics": "amor",
    "complexitat": "baixa",
    "caracteristiques": "simples",
    "desenvolupament_del_personatge": "alt",
    "accio_o_reflexio": "accio",
    "longitud": "mitjana",
    "epoca": "actual",
    "detall_cientific": "baix"
    "llibres_recomenats": ["llibre4", "llibre6", "llibre7"]
    "puntuacio_llibres": [4, 4, 1]
}
user = Usuari(1, att)

att2 = {
    "edat": 50,
    "estil_literari": "simbolisme",
    "temas_especifics": "terror",
    "complexitat": "alta",
    "caracteristiques": "complexes",
    "desenvolupament_del_personatge": "mitja",
    "accio_o_reflexio": "reflexio",
    "longitud": "llarga",
    "epoca": "futura",
    "detall_cientific": "alta"
    "llibres_recomenats": ["llibre1", "llibre2", "llibre3", "llibre4"]
    "puntuacio_llibres": [4, 1, 5, 3]
}
user2 = Usuari(2, att2)

att3 = {
    "edat": 50,
    "estil_literari": "simbolisme",
    "temas_especifics": "terror",
    "complexitat": "baixa",
    "caracteristiques": "complexes",
    "desenvolupament_del_personatge": "mitja",
    "accio_o_reflexio": "reflexio",
    "longitud": "llarga",
    "epoca": "futura",
    "detall_cientific": "alta"
    "llibres_recomenats": []
    "puntuacio_llibres": []
}
user3 = Usuari(3, att3)
print(user)

User 1
edat: 20
estil_literari: realisme
temas_especifics: amor
complexitat: baixa
caracteristiques: simples
desenvolupament_del_personatge: alt
accio_o_reflexio: accio
longitud: mitjana
epoca: actual
detall_cientific: baix



In [5]:
cbr = CBR([user, user2])
encoder = cbr.get_encoder()
users = cbr.transform_user_to_numeric(encoder, cbr.get_users())
